In [11]:
# Install CUDA C++ plugin for Colab:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [12]:
# Detect selected GPU and its NVIDA architecture:
import subprocess
gpu_info = subprocess.getoutput("nvidia-smi --query-gpu=name,compute_cap --format=csv,noheader,nounits")
if "not found" in gpu_info.lower(): raise RuntimeError("Error: No GPU found. Please select a GPU runtime environment.")
gpu_name, compute_cap = map(str.strip, gpu_info.split(','))
gpu_arch = f"sm_{compute_cap.replace('.', '')}"

print(f"{'GPU Name':<15}: {gpu_name}")
print(f"{'Architecture':<15}: {gpu_arch}")

GPU Name       : Tesla T4
Architecture   : sm_75


In [13]:
%%cuda -c "--gpu-architecture sm_80"
#include <stdio.h>

#include <stdio.h>
#include <cuda_runtime.h>


__global__ void bitonic_stage(int *data, int n, int i, int j){
    int k = blockIdx.x * blockDim.x + threadIdx.x;
    if (k >= n) return;

    // size = 2^i, stride = 2^(j-1)
    int size   = 1 << i;
    int stride = 1 << (j - 1);

    // participation: only threads in the lower 'stride' part of each size-block act
    if ((k & (size - 1)) >= stride) return;

    // correct partner pairing (XOR with stride)
    int partner = k ^ stride;
    if (partner >= n) return;

    int region    = k >> i;             // = k / size
    int ascending = ((region & 1) == 0);

    int a = data[k];
    int b = data[partner];

    if (ascending) {
        if (a > b) {
            data[k]       = b;
            data[partner] = a;
        }
    } else {
        if (a < b) {
            data[k]       = b;
            data[partner] = a;
        }
    }
}


void bitonic_sort_gpu(int *h_arr, int n){
    int *d_arr;
    cudaMalloc(&d_arr, n * sizeof(int));
    cudaMemcpy(d_arr, h_arr, n * sizeof(int), cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks  = (n + threads - 1) / threads;

    // steps = log2(n) for n power of 2
    int steps = 0;
    for (int tmp = n; tmp > 1; tmp >>= 1) {
        steps++;
    }

    for (int i = 1; i <= steps; i++) {
        for (int j = i; j >= 1; j--) {
            bitonic_stage<<<blocks, threads>>>(d_arr, n, i, j);
            cudaDeviceSynchronize();
        }
    }

    cudaMemcpy(h_arr, d_arr, n * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_arr);
}


int main(){

    // Array size MUST be a power of 2 (here 8)
    int arr[] = {3, 7, 8, 9, 5, 4, 3, 2};
    int n = sizeof(arr) / sizeof(arr[0]);

    printf("Unsorted: ");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    bitonic_sort_gpu(arr, n);

    printf("Sorted:   ");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    return 0;
}


Unsorted: 3 7 8 9 5 4 3 2 
Sorted:   3 7 8 9 5 4 3 2 



In [14]:
%%cuda -c "--gpu-architecture sm_80"
#include <stdio.h>

#include <stdio.h>
#include <cuda_runtime.h>


__global__ void bitonic_stage(int *data, int n, int i, int j){

    int k = blockIdx.x * blockDim.x + threadIdx.x;
    if (k >= n) return;

    // 'i' corresponds to the log2 of the current bitonic sequence length (2^i)
    // 'j' corresponds to the log2 of the comparison distance (2^(j-1))

    int current_len = 1 << i;          // The length of the current bitonic sequence block
    int comp_dist   = 1 << (j - 1);    // The comparison distance

    // Determine the sorting direction for this 'current_len'-sized block
    // If k is in an even 'current_len'-sized block, sort ascending. Else descending.
    int ascending = ((k / current_len) % 2 == 0); // Equivalent to ( (k >> i) & 1 ) == 0

    // This thread only participates if 'k' is in the first half of a 'comp_dist * 2' segment.
    if ((k % (comp_dist * 2)) >= comp_dist) return;

    int partner = k + comp_dist;
    if (partner >= n) return;

    int a = data[k];
    int b = data[partner];

    if (ascending) {
        if (a > b) {
            data[k]       = b;
            data[partner] = a;
        }
    } else {
        if (a < b) {
            data[k]       = b;
            data[partner] = a;
        }
    }
}


void bitonic_sort_gpu(int *h_arr, int n)
{
    int *d_arr;
    cudaMalloc(&d_arr, n * sizeof(int));
    cudaMemcpy(d_arr, h_arr, n * sizeof(int), cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks  = (n + threads - 1) / threads;

    // steps = log2(n) for n power of 2
    int steps = 0;
    for (int tmp = n; tmp > 1; tmp >>= 1) {
        steps++;
    }

    for (int i = 1; i <= steps; i++) {
        for (int j = i; j >= 1; j--) {
            bitonic_stage<<<blocks, threads>>>(d_arr, n, i, j);
            cudaDeviceSynchronize();
        }
    }

    cudaMemcpy(h_arr, d_arr, n * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_arr);
}


int main(){

    // Array size MUST be a power of 2 (here 8)
    int arr[] = {3, 7, 8, 9, 5, 4, 3, 2};
    int n = sizeof(arr) / sizeof(arr[0]);

    printf("Unsorted: ");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    bitonic_sort_gpu(arr, n);

    printf("Sorted:   ");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    return 0;
}


Unsorted: 3 7 8 9 5 4 3 2 
Sorted:   3 7 8 9 5 4 3 2 

